In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [2]:
# загрузка датасетов main_gold, в котором собраны все данные по ценам по конкурентам и данные нашей сети и movement с динамикой, а также датасета с распределением по диапазонам цен

In [33]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

colors_shop = {
    '585*Золотой':'rgb(229, 134, 6)',
    'Sunlight':'rgb(93, 105, 177)',
    'Sokolov':'rgb(82, 188, 163)',
    '585Gold':'rgb(153, 201, 69)',
    'Линии любви':'rgb(204, 97, 176)',
    '585 Gold':'rgb(153, 201, 69)'
}
color_tn = dict(zip([
    s for s in main_gold['tn'].unique()], [c for c in px.colors.qualitative.Vivid[:(len(main_gold['tn'].unique()) + 1)]]))

SIDESTYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "18rem",
    "padding": "2rem 1rem",
    "background-color": "#BDCBEA",
}


CONTSTYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}


PAGE_SIZE = 10


app.layout = html.Div([
    dcc.Location(id="url"),
    html.Div(
        [  html.Img(className="fit-picture",
             src="https://www.585zolotoy.ru/_nuxt/img/585-logo.1ebf474.svg",
            width="180", height="111",
             alt="585 Золотой"),
            html.H2("Меню", className="display-3", style={'color': 'white'}),
            html.Hr(style={'color': 'white'}),
            dbc.Nav(
                [
                    dbc.NavLink("Количество изделий", href="/page1", active="exact", style={'color': 'white'}),
                    dbc.NavLink("Цены", href="/page2", active="exact", style={'color': 'white'}),
                    dbc.NavLink("Диапазоны", href="/page3", active="exact", style={'color': 'white'})
                ],
                vertical=True,pills=True),
        ],
        style=SIDESTYLE,
    ),
    html.Div(id="page-content", children=[], style=CONTSTYLE)
])

@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")])

def pagecontent(pathname):
    if pathname == "/page1":
        return [
            html.Div(
                children=[
                    html.H1(children='Онлайн мониторинг конкурентов', className='header-title', style={'textAlign': 'center'}),
                    html.Br(),
                    html.H4(
                        children='Распределение артикулов по ТН', className='header-description', style={'textAlign': 'center'}
                    )
                ], className='header', style={'border-width': '5px',
                                              'padding-left': '90px',
                                              'width':'1200px'}),
            html.Br(),
 
            html.Div([
                html.Div([
                    html.Label('Товарное направление'),
                    dcc.Dropdown(
                        options=[{'label':c, 'value':c} for c in main_gold['tn'].unique()],
                        value=['БК', 'ИФ', 'ЦБ'],
                        multi=True,
                        id='tn', style={'width':510}
                    ), 
                    dcc.RadioItems(
                        options=[{'label':'Все', 'value':'All'},
                                 {'label':'БК, ЦБ, ИФ', 'value':'W'},
                                 {'label':'ДК, ПДК', 'value':'D'},
                                 {'label':'Выбранные', 'value':'C'}],
                        value='C',
                        id='btn_tn', 
                        labelStyle={
                            'display': 'inline-block', 'width': '11.5%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 5
                        }
                    ),
                    dcc.Graph(id = 'all_prod'),
                    dcc.Graph(id = 'all_table'),
            ], 
               style={'border-style':'solid',
                      'border-color':'#BDCBEA',
                      'border-width':'5px',
                      'padding':'10px',
                      'width':'1200px',
                     }),], style={'padding-left': '70px',}),
            html.Br()]
    
    elif pathname == "/page2":
        return [
            html.Div(
                children=[
                    html.H1(children='Онлайн мониторинг конкурентов', className='header-title', style={'textAlign': 'center'}),
                    html.Br(),
                    html.H4(
                        children='Сравнение цен', className='header-description', style={'textAlign': 'center'}
                    )
                ], className='header', style={'border-width': '5px',
                                              'padding-left': '90px',
                                              'width':'1200px'}),
            html.Br(),
            
            
            html.Div([
                html.Div([
                    dcc.RadioItems(
                        options=[{'label':'БК, ЦБ, ИФ', 'value':'W'},
                                 {'label':'ДК, ПДК', 'value':'D'}],
                        value='W',
                        id = 'tn',
                        labelStyle={
                            'display': 'inline-block', 'width': '11.5%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 5
                        }
                    ),
                    dcc.RadioItems(
                        options=[{'label':'Минимальная цена', 'value':'min'},
                                 {'label':'Средняя цена', 'value':'mean'},
                                 {'label':'Максимальная цена', 'value':'max'}],
                        value='mean',
                        id = 'func',
                        labelStyle={
                            'display': 'inline-block', 'width': '20%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 5
                        }
                    ),
                    dcc.Graph(id = 'table_prices'),
                    dcc.Graph(id = 'movement')
            ], 
               style={'border-style':'solid',
                      'border-color':'#BDCBEA',
                      'border-width':'5px',
                      'padding':'10px',
                      'width':'1200px',
                     }),], style={'padding-left': '70px',}),
            html.Br(),
        ]
    
    
    elif pathname == "/page3":
        return [
            html.Div(
                children=[
                    html.H1(children='Онлайн мониторинг конкурентов', className='header-title', style={'textAlign': 'center'}),
                    html.Br(),
                    html.H4(
                        children='Распределение артикулов по диапазонам цен', className='header-description', style={'textAlign': 'center'}
                    )
                ], className='header', style={'border-width': '5px',
                                              'padding-left': '90px',
                                              'width':'1200px'}),
            html.Br(),
            
            
            html.Div([
                html.Div([                    
                    dcc.RadioItems(
                        options=[{'label':'БК, ЦБ, ИФ', 'value':'W'},
                                 {'label':'ДК, ПДК', 'value':'D'}],
                        value='W',
                        id = 'tn',
                        labelStyle={
                            'display': 'inline-block', 'width': '11.5%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 5
                        }
                    ),
                    dcc.RadioItems(
                        options=[{'label':'Абсолютные значения', 'value':'abs'},
                                 {'label':'Относительные значения', 'value':'ratio'}],
                        value='ratio',
                        id = 'btn_val',
                        labelStyle={
                            'display': 'inline-block', 'width': '20%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 5
                        }
                    ),
                    dcc.Graph(id = 'diap'),
            ], 
               style={'border-style':'solid',
                      'border-color':'#BDCBEA',
                      'border-width':'5px',
                      'padding':'10px',
                      'width':'1200px',
                     }),], style={'padding-left': '70px',}),
            html.Br()]
    
@app.callback(
     Output('all_prod', 'figure'),
     Input('tn', 'value'),
     Input('btn_tn', 'value')
)

def update_all_prod(tn, btn_tn):
    if btn_tn == 'All':
        tn = list(main_gold['tn'].unique())
    elif btn_tn == 'W':
        tn = ['БК', 'ИФ', 'ЦБ']
    elif btn_tn == 'D':
        tn = ['ДК', 'ПДК']
    else:
        tn = tn
    pivot_sum = main_gold[main_gold['tn'].isin(tn)].pivot_table(
        index='tn', columns='shop', values='article', aggfunc='count').round(0).astype('int')
    fig_all_prod = go.Figure()
    for i in pivot_sum.index:
        fig_all_prod.add_trace(go.Bar(
            x=pivot_sum.columns,
            y=pivot_sum.loc[i, :],
            marker_color=color_tn[i],
            name=i,textposition = "auto",
            texttemplate = "%{y:.0f}"
        ))
    fig_all_prod.update_layout(barmode='relative',
                               height=500,
                               plot_bgcolor='white',
                               legend_orientation="v",
                               xaxis_tickangle=-45,
                               title=dict(
                                   text="Распределение артикулов по ТН, шт.",
                                   y=.85,
                                   x=0.45,
                                   xanchor='center',
                               yanchor='auto'))
    fig_all_prod.update_yaxes(tickformat='%{y:.2f}')
    return fig_all_prod

@app.callback(
     Output('all_table', 'figure'),
     Input('tn', 'value'),
     Input('btn_tn', 'value')
)

def all_table(tn, btn_tn):
    if btn_tn == 'All':
        tn = list(main_gold['tn'].unique())
    elif btn_tn == 'W':
        tn = ['БК', 'ИФ', 'ЦБ']
    elif btn_tn == 'D':
        tn = ['ДК', 'ПДК']
    else:
        tn = tn
    result = main_gold[main_gold['tn'].isin(list(tn))].pivot_table(
        columns='shop', values='article', aggfunc='count'
    ).round(0)
    fig_all_table = go.Figure([go.Table(header = {'values': [''] + [f'<b>{i}</b>' for i in result.columns],
                                         'fill_color': 'lightgrey',
                                         'align': 'center'},
                               cells = {'values': ['Итого'] + [
                                   ['{:,d}'.format(num).replace(',', ' ') for num in row] for row in result.T.values]})])
    fig_all_table.update_layout(
        autosize=False,
        width=1150,
        height=255,
        title=dict(
            text=f'Количество изделий {", ".join(tn)}, шт.',
            y=.9,
            x=0.45,
            xanchor='center',
            yanchor='top')
    )
    return fig_all_table    

    
@app.callback(
     Output('table_prices', 'figure'),
     Input('tn', 'value'),
     Input('func', 'value')
     )

def update_table_prices(tn, func):
    pr_disp = {'min':'Минимальная',
               'mean':'Средняя',
               'max':'Максимальная'}
    if tn == 'W':
        main_bk = main_gold.copy()[main_gold['tn'].isin(['БК', 'ИФ', 'ЦБ'])]
        fig_table_prices = go.Figure()
        for i in main_bk['shop'].unique():
            fig_table_prices.add_trace(go.Bar(x=list(main_bk['tn'].unique()) + ['Итого'],
                                              y=main_bk[
                                                  main_bk['shop'] == i
                                              ].groupby('tn')['price_per_gram'].agg(func).round().astype('int').append(
                                                  pd.Series(round(main_bk[main_bk['shop'] == i]['price_per_gram'].agg(func)))), 
                                              name=i,
                                              marker_color=colors_shop[i],
                                              textposition = "outside",
                                              texttemplate = "%{y:.0f}"))
        fig_table_prices.update_layout(legend_orientation="h",
                                       height=500,
                                       plot_bgcolor='white',
                                       yaxis={'visible': False, 'showticklabels': False},
                                       title=dict(
                                           text=f'{pr_disp[func]} цена за грамм, руб.',
                                           y=.9,
                                           x=0.45,
                                           xanchor='center',
                                           yanchor='top'))
    else:
        main_dk = main_gold.copy()[main_gold['tn'].isin(['ДК', 'ПДК'])]
        fig_table_prices = go.Figure()
        for i in main_dk['shop'].unique():
            fig_table_prices.add_trace(go.Bar(x=list(main_dk['tn'].unique()), 
                                              y=main_dk[
                                                  main_dk['shop'] == i
                                              ].groupby('tn')['price_discount'].agg(func).round().astype('int'), 
                                              name=i,
                                              marker_color=colors_shop[i],
                                              textposition = "outside",
                                              texttemplate = "%{y:.0f}"))
        fig_table_prices.update_layout(legend_orientation="h", 
                                       height=500,
                                       plot_bgcolor='white',
                                       yaxis={'visible': False, 'showticklabels': False},
                                       title=dict(
                                           text=f'{pr_disp[func]} цена за изделие, руб.',
                                           y=.9,
                                           x=0.45,
                                           xanchor='center',
                                           yanchor='top'))
    return fig_table_prices    

@app.callback(
     Output('movement', 'figure'),
     Input('tn', 'value'),
     Input('func', 'value')
     )
def update_movement(tn, func):
    if tn == 'W' and func == 'mean':
        fig_movement = go.Figure()
        for i in movement.index:
            fig_movement.add_trace(go.Scatter(x=movement.columns,
                                              y=movement.loc[i, :],
                                              name=i,
                                              text=movement.loc[i, :],
                                              mode='lines+markers+text',
                                              textposition = "top right",
                                              marker_color=colors_shop[i],
                                              texttemplate = "%{y:.0f}"
                                             ))
            fig_movement.update_layout(plot_bgcolor='white',
                                       legend_orientation="v",
                                       autosize=False,
                                       width=1100,
                                       height=500,
                                       xaxis_tickangle=-45,
                                       title=dict(
                                           text="Динамика средней цены за грамм золота среди весовых изделий, руб.",
                                           y=0.85,
                                           x=0.45,
                                           xanchor='center',
                                           yanchor='top'
                                       ))
    else:
        fig_movement = go.Figure()
        fig_movement.update_layout(plot_bgcolor='white',
                                   autosize=False,
                                   width=300,
                                   height=10,
                                   yaxis={'visible': False, 'showticklabels': False},
                                   xaxis={'visible': False, 'showticklabels': False},
                                  )
    return fig_movement


@app.callback(
     Output('diap', 'figure'),
     Input('tn', 'value'),
     Input('btn_val', 'value')
     )


def update_diap(tn, btn_val):
    fig_diap = go.Figure()
    if tn == 'W':
        diap_df = main_gold.copy()[(main_gold['tn'].isin(['БК', 'ИФ', 'ЦБ']))&(main_gold['price_per_gram'].notna())]
        diap_df['price_case'] = pd.cut(diap_df['price_per_gram'],
                                       [0, 4000, 4500, 5000, 5500, 6000, 7000, 1000000],
                                       labels=['до 4 000 руб.', '4 000 руб. - 4 500 руб.', '4 500 руб. - 5 000 руб.', 
                                               '5 000 руб. - 5 500 руб.', '5 500 руб. - 6 000 руб.', 
                                               '6 000 руб. - 7 000 руб.', 'свыше 7 000 руб.'])
        pivot_diap = diap_df.pivot_table(index='price_case', columns='shop', values='article', 
                                         aggfunc='count', margins=True).drop('All', axis=1)
        
        
        diap_bk_color = dict(zip(
            [d for d in diap_df['price_case'].unique()], 
            [c for c in px.colors.qualitative.Vivid[:len(diap_df['price_case'].unique()) + 1]]
        ))
        if btn_val == 'abs':
            for i in pivot_diap.index[:-1]:
                fig_diap.add_trace(go.Bar(
                    x=pivot_diap.columns,
                    y=pivot_diap.loc[i, :],
                    marker_color=diap_bk_color[i],
                    name=i,textposition = "auto",
                    texttemplate = "%{y:.0f}"
                ))
            fig_diap.update_layout(barmode='relative', 
                                   height=500,
                                   plot_bgcolor='white',
                                   legend_orientation="v",
                                   xaxis_tickangle=-45,
                                   title=dict(
                                       text="Распределение артикулов по диапазонам цены за грамм",
                                       y=0.85,
                                       x=0.45,
                                       xanchor='center',
                                   yanchor='top'))
            fig_diap.update_yaxes(tickformat='%{y:.2f}')
        elif btn_val == 'ratio':
            pivot_diap_ratio = pivot_diap.copy()
            for i in list(pivot_diap_ratio.index[:-1]):
                for j in list(pivot_diap_ratio.columns):
                    pivot_diap_ratio.loc[i, j] = pivot_diap_ratio.loc[i, j] / pivot_diap_ratio.loc['All', j]            
            fig_diap = go.Figure()
            for i in pivot_diap_ratio.index[:-1]:
                fig_diap.add_trace(go.Bar(
                    x=pivot_diap_ratio.columns,
                    y=pivot_diap_ratio.loc[i, :],
                    marker_color=diap_bk_color[i],
                    name=i,
                    textposition = "auto",
                    texttemplate = "%{y:.0%}"
                ))
            fig_diap.update_layout(barmode='relative', 
                                   height=500,
                                   plot_bgcolor='white',
                                   legend_orientation="v",
                                   xaxis_tickangle=-45,
                                   yaxis={'visible': False, 'showticklabels': False},
                                   title=dict(
                                       text="Распределение артикулов по диапазонам цены за грамм",
                                       y=0.85,
                                       x=0.45,
                                       xanchor='center',
                                   yanchor='top'))
            fig_diap.update_yaxes(tickformat='%{y:.2f}')
    else:
        diap_dk = main_gold.copy()[main_gold['tn'].isin(['ДК', 'ПДК'])]
        diap_dk['price_case'] = pd.cut(diap_dk['price_discount'],
                                       [0, 10000, 15000, 20000, 25000, 30000, 10000000],
                                       labels=['до 10 000 руб.', '10 000 руб. - 15 000 руб.', '15 000 руб. - 20 000 руб.', 
                                               '20 000 руб. - 25 000 руб.', '25 000 руб. - 30 000 руб.', 'свыше 30 000 руб.'])
        pivot_diap_dk = diap_dk.pivot_table(index='price_case', columns='shop', values='article', 
                                            aggfunc='count', margins=True).drop('All', axis=1)
        diap_dk_color = dict(zip(
            [d for d in diap_dk['price_case'].unique()], 
            [c for c in px.colors.qualitative.Vivid[:len(diap_dk['price_case'].unique()) + 1]]
        ))
        if btn_val == 'abs':
            for i in pivot_diap_dk.index[:-1]:
                fig_diap.add_trace(go.Bar(
                    x=pivot_diap_dk.columns,
                    y=pivot_diap_dk.loc[i, :],
                    marker_color=diap_dk_color[i],
                    name=i,textposition = "auto",
                    texttemplate = "%{y:.0f}"
                ))
            fig_diap.update_layout(barmode='relative', 
                                   height=500,
                                   plot_bgcolor='white',
                                   legend_orientation="v",
                                   xaxis_tickangle=-45,
                                   title=dict(
                                       text="Распределение артикулов по диапазонам цены за изделие",
                                       y=0.85,
                                       x=0.45,
                                       xanchor='center',
                                   yanchor='top'))
            fig_diap.update_yaxes(tickformat='%{y:.2f}')
        elif btn_val == 'ratio':
            pivot_diap_dk_ratio = pivot_diap_dk.copy()
            for i in list(pivot_diap_dk_ratio.index[:-1]):
                for j in list(pivot_diap_dk_ratio.columns):
                    pivot_diap_dk_ratio.loc[i, j] = pivot_diap_dk_ratio.loc[i, j] / pivot_diap_dk_ratio.loc['All', j]
            for i in pivot_diap_dk_ratio.index[:-1]:
                fig_diap.add_trace(go.Bar(
                    x=pivot_diap_dk_ratio.columns,
                    y=pivot_diap_dk_ratio.loc[i, :],
                    marker_color=diap_dk_color[i],
                    name=i,
                    textposition = "auto",
                    texttemplate = "%{y:.0%}"
                ))
            fig_diap.update_layout(barmode='relative',
                                   height=500,
                                   plot_bgcolor='white',
                                   legend_orientation="v",
                                   xaxis_tickangle=-45,
                                   yaxis={'visible': False, 'showticklabels': False},
                                   title=dict(
                                       text="Распределение артикулов по диапазонам цены за изделие",
                                       y=0.85,
                                       x=0.45,
                                       xanchor='center',
                                   yanchor='top'))
            fig_diap.update_yaxes(tickformat='%{y:.0%}')

    return fig_diap

app.run_server(mode='jupyterlab', host = '10.167.12.219', debug=True, use_reloader=False)